In [30]:
from pyspark.sql import SparkSession

In [31]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [33]:
spark.stop()

In [35]:
from pyspark import SparkConf, SparkContext

In [36]:
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('spark-basic')
sc = SparkContext(conf = conf)

In [37]:
sc

<SparkContext master=local appName=spark-basic>

In [8]:
import os
os.chdir('..')

In [42]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Word Count") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [43]:
spark

In [11]:
os.getcwd()

'/Users/ksunil/Desktop/Karan/Codebase'

In [58]:
def get_depth_predictions(reef_name):
    depth_predictions_fp = os.path.join('data',reef_name, 'Output', 'Depth Predictions','CSV_files')
    cwd = os.getcwd()
    for i,f in enumerate(os.listdir(depth_predictions_fp)):
        if f == '.DS_Store': continue
        
        data_fp = os.path.join(depth_predictions_fp, f)
        fp = 'file://' + cwd + '/' + data_fp
        if i == 1:
            temp = spark.read.csv(fp,header = True, inferSchema = True)
            df = temp
        else:
            temp = spark.read.csv(fp,header = True, inferSchema = True)
            df = df.unionAll(temp)
    return df


In [59]:
moce = get_depth_predictions('Moce')

In [61]:
len(moce)

TypeError: object of type 'DataFrame' has no len()

In [62]:
moce.count()

IllegalArgumentException: 'Unsupported class file major version 55'

In [66]:
df = moce
print((df.count(), len(df.columns)))

IllegalArgumentException: 'Unsupported class file major version 55'